In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [3]:
# Connect to a local SQLite database. SQLite is a simple file oriented database management "library." 
# in the following we launch a SQLite database stored in the .sqlite file.
dbfile = Path("ZMainBldg.sqlite")
#if dbfile.exists():
    #dbfile.unlink()

sqlite_uri = "sqlite:///ZMainBldg.sqlite"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

In [4]:
# CREATE TABLE alameda AS SELECT * FROM main_bldg WHERE County='ALAMEDA';
sqlite_uri = "sqlite:///ZCountyExtracts.sqlite"
zce_engine = sqlalchemy.create_engine(sqlite_uri)

In [5]:
# Check tables
sqlite_engine.table_names()

['main_bldg', 'sample']

In [6]:
# Work on sample table for now. sample contains a Simple Random Sample of main_bldg of size 100,000
sql_expr = """
SELECT * FROM sample
GROUP BY ImportParcelID;
"""
sample = pd.read_sql(sql_expr, sqlite_engine)
sample.head()

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode
0,659D952A-45D5-E511-80C1-3863BB43AC67,8904758,06001,CA,ALAMEDA,2015-07-01,072015,23,BKF,1-145-23,,1,A,,,,P,373,,,4TH,ST,,373 4TH ST,OAKLAND,CA,94607,,,,,,,,11033.3,2014,,,,17-022,1,,,,,POCKET BUILDING H O A,,,3D,,,,,,,,,,,,,,,,1,0.189,8234,,,,LE,1791473059,Y,APT,3D,C027,Y,37.7966,-122.273,060019832.001041,,,,,,,,0,361491,-1,659D952A-45D5-E511-80C1-3863BB43AC67,10,O,CONDOMINIUM,103,RR106,,,1,,C,,,,1909,1971,,3,1,0,,2,,0,,,C,0,,,,,,Y,,,,,,,,,
1,D53CFB02-2AD5-E511-80C1-3863BB43AC67,8905243,06001,CA,ALAMEDA,,082010,18,BKF,1-157-75,,1,,,,,,206,,,2ND,ST,,206 2ND ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,LE,339051770,Y,,,C010,Y,37.7941,-122.271,060014033.001032,,,,,,,,0,330739,-1,D53CFB02-2AD5-E511-80C1-3863BB43AC67,,O,"MULTI, CONDOMINIUMS",7300,RR106,,,1,,,,,,2006,2006,,5,3,1,,,,,,1,S,,,,,,,,,,,,,,,,
2,95ACC9E6-CDD5-E511-80C1-3863BB43AC67,8905265,06001,CA,ALAMEDA,,072013,21,BKF,1-157-95,,1,A,,,,,200,,,2ND,ST,,200 2ND ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,,,,,LE,340625705,Y,UNIT,504,C010,Y,37.7939,-122.271,060014033.001026,,,,,,,,0,332927,-1,95ACC9E6-CDD5-E511-80C1-3863BB43AC67,75,,CONDOMINIUMS - SINGLE RESIDENTIAL LIVING UNIT,7300,RR106,,,1,,B,,,,2006,2006,,5,4,2,,,,,,1,S,,,,,,,,,,,,,,,,
3,807471F8-66D5-E511-80C1-3863BB43AC67,8905429,06001,CA,ALAMEDA,,082002,10,BKF,001 -0179-020,,1,,,,,,228,,,7TH,ST,,228 7TH ST,OAKLAND,CA,94607,,,,,,,,,,,,,17-022,,,,,,,,,,,,,,,,,,,,,,,,,1,,2500,,,,,334985366,Y,,,C010,Y,37.7

In [7]:
# 100000
len(sample)

99503

In [8]:
# Number of sample Alameda records
len(sample[sample['County'] == 'ALAMEDA'])

3840

In [9]:
# Get cities from JC RC Inventory
cities = list(pd.read_excel('jc_rc.xlsx')['Unnamed: 0'].values[1:-1])
cities

['Alameda',
 'Berkeley',
 'Beverly Hills',
 'East Palo Alto',
 'Emeryville',
 'Glendale',
 'Hayward',
 'Los Angeles',
 'Maywood',
 'Mountain View',
 'Oakland',
 'Richmond',
 'San Diego',
 'San Francisco',
 'San Jose',
 'Santa Monica',
 'Union City',
 'West Hollywood',
 'Palm Springs',
 'Los Gatos',
 'Thousand Oaks']

In [10]:
zce_engine.table_names()

['alameda', 'alameda_u', 'los_angeles', 'los_angeles_u']

In [13]:
# Load CHPC which will help determine if property is subsidized
chpc = pd.read_excel('chpc.xls', sheet_name='Cleaned')

In [14]:
chpc.head(1)

,DataSet,Property Name,Address_Cleaned,Units with Assistance (Inclusive of All Programs),Units with Rental Assistance (HUD and/or mfhd),HUD Program?,Application Number (TCAC),Type of Tax Credit (TCAC),Application Stage (TCAC),Placed in Service (PIS) Date (TCAC),Construction Type (TCAC),Housing Type (TCAC),Low Income Units (TCAC),Number of SRO/Studio Units,Number of 1 Bedroom Units,Number of 2 Bedroom Units,Number of 3 Bedroom Units,Number of 4 Bedroom Units,Number of 5 Bedroom Units,Number of 6 Bedroom Units,Units at or below 30% AMI,Units at 35% AMI,Units at 40% AMI,Units at 45% AMI,Units at 50% AMI,Units at 55% AMI,Units at 60% AMI,General Partner (1),Management Company,Developer,Borrower+Project ID (mfhd),Tax_Credit_Indicator (mfhd),Date_Tax_Credit_Expires (mfhd),Date_Of_Operation (mfhd),Date_Restrictive_Clause_Expires (mfhd),Notes,Combined Multiple Sites?,LatLong_Google,Latitude,Longitude,Unique: Property Name_CleanedAddress-,GEOID (from Geocode of Lat/Long),TRACTCE (from Geocode of Lat/Long),Within Half Mile of HQT?
0,"HUD, TCAC",COMM22 Senior Housing,"690 Beardsley Street, San Diego, California, 92113",69.0,NaN,NaN,CA-2013-852,0.04,Preliminary Reservation,2015,New Construction,Seniors,69.0,15.0,51.0,4.0,0.0,0.0,0.0,0.0,15.0,0.0,12.0,0.0,42.0,0.0,0.0,"COMM22 Senior GP, LLC",BRIDGE Property Management Company,BRIDGE Housing Corporation,NaN,NaN,NaN,NaN,NaN,HUD has 30 units,NaN,"32.7045781,-117.1444356",32.704578,-117.144436,"COMM22 Senior Housing690 Beardsley Street, San Diego, California, 92113",6.073005e+09,4900.0,1.0


In [15]:
# Compare Address numbers and name 
chpc['Address_Num'] = chpc['Address_Cleaned'].str.extract(r'(\d+)')
chpc['Address_Name'] = chpc['Address_Cleaned'].str.extract(r'([^\d,-]\w+)').str.upper()
chpc.head(1)

/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,DataSet,Property Name,Address_Cleaned,Units with Assistance (Inclusive of All Programs),Units with Rental Assistance (HUD and/or mfhd),HUD Program?,Application Number (TCAC),Type of Tax Credit (TCAC),Application Stage (TCAC),Placed in Service (PIS) Date (TCAC),Construction Type (TCAC),Housing Type (TCAC),Low Income Units (TCAC),Number of SRO/Studio Units,Number of 1 Bedroom Units,Number of 2 Bedroom Units,Number of 3 Bedroom Units,Number of 4 Bedroom Units,Number of 5 Bedroom Units,Number of 6 Bedroom Units,Units at or below 30% AMI,Units at 35% AMI,Units at 40% AMI,Units at 45% AMI,Units at 50% AMI,Units at 55% AMI,Units at 60% AMI,General Partner (1),Management Company,Developer,Borrower+Project ID (mfhd),Tax_Credit_Indicator (mfhd),Date_Tax_Credit_Expires (mfhd),Date_Of_Operation (mfhd),Date_Restrictive_Clause_Expires (mfhd),Notes,Combined Multiple Sites?,LatLong_Google,Latitude,Longitude,Unique: Property Name_CleanedAddress-,GEOID (from Geocode of Lat/Long),TRACTCE (from Geocode of Lat/Long),Within Half Mile of HQT?,Address_Num,Address_Name
0,"HUD, TCAC",COMM22 Senior Housing,"690 Beardsley Street, San Diego, California, 92113",69.0,NaN,NaN,CA-2013-852,0.04,Preliminary Reservation,2015,New Construction,Seniors,69.0,15.0,51.0,4.0,0.0,0.0,0.0,0.0,15.0,0.0,12.0,0.0,42.0,0.0,0.0,"COMM22 Senior GP, LLC",BRIDGE Property Management Company,BRIDGE Housing Corporation,NaN,NaN,NaN,NaN,NaN,HUD has 30 units,NaN,"32.7045781,-117.1444356",32.704578,-117.144436,"COMM22 Senior Housing690 Beardsley Street, San Diego, California, 92113",6.073005e+09,4900.0,1.0,690,BEARDSLEY


In [16]:
# Merge based on euclidean distance
chpc = chpc.dropna(subset=['Longitude', 'Latitude'])
def merge_euclidean(table, eps=0.01):
    chpc_list = []
    table['merge_row'] = table.index.values
    for i, row in table.iterrows():
        chpc_subset = chpc.loc[((chpc.Longitude - row.PropertyAddressLongitude)**2 + (chpc.Latitude - row.PropertyAddressLatitude)**2 < eps) & (chpc.Address_Num == row.PropertyHouseNumber)]
        chpc_subset['merge_row'] = i
        chpc_list.append(chpc_subset)
    chpc_found = pd.concat(chpc_list)

    result = pd.merge(table, chpc_found, on='merge_row', how='left')
    return result

In [17]:
# Fix dtype for merging
sample.to_csv('sample.csv', sep='|', index=False)
sample = pd.read_csv('sample.csv', sep='|')

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,12,16,17,26,30,33,40,41,42,43,46,48,49,50,51,52,55,60,61,63,69,70,79,94,97,99,100,102,103,118,119,120,121,122,123,124,127,128,129,130,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
a